In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\admin\\Desktop\\Predictive Maintenance\\Predictive_Maintenance_With_MLops\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\admin\\Desktop\\Predictive Maintenance\\Predictive_Maintenance_With_MLops'

In [5]:
# data validation 

import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_cleaning/train.csv")
data.head()

,engine,cycle,setting_1,setting_2,(LPC outlet temperature) (◦R),(LPT outlet temperature) (◦R),(bypass-duct pressure) (psia),(HPC outlet pressure) (psia),(Physical core speed) (rpm),(HPC outlet Static pressure) (psia),(Ratio of fuel flow to Ps30) (pps/psia),(Bypass Ratio),(Bleed Enthalpy),(High-pressure turbines Cool air flow),(Low-pressure turbines Cool air flow),RUL
0,1,2,0.0019,-0.0003,642.15,1403.14,21.61,553.75,9044.07,47.49,522.28,8.4318,392,39.00,23.4236,190
1,1,3,-0.0043,0.0003,642.35,1404.20,21.61,554.26,9052.94,47.27,522.42,8.4178,390,38.95,23.3442,189
2,1,4,0.0007,0.0000,642.35,1401.87,21.61,554.45,9049.48,47.13,522.86,8.3682,392,38.88,23.3739,188
3,1,5,-0.0019,-0.0002,642.37,1406.22,21.61,554.00,9055.15,47.28,522.19,8.4294,393,38.90,23.4044,187
4,1,6,-0.0043,-0.0001,642.10,1398.37,21.61,554.67,9049.68,47.16,521.68,8.4108,391,38.98,23.3669,186


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20630 entries, 0 to 20629
Data columns (total 16 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   engine                                   20630 non-null  int64  
 1   cycle                                    20630 non-null  int64  
 2   setting_1                                20630 non-null  float64
 3   setting_2                                20630 non-null  float64
 4   (LPC outlet temperature) (◦R)            20630 non-null  float64
 5   (LPT outlet temperature) (◦R)            20630 non-null  float64
 6   (bypass-duct pressure) (psia)            20630 non-null  float64
 7   (HPC outlet pressure) (psia)             20630 non-null  float64
 8   (Physical core speed) (rpm)              20630 non-null  float64
 9   (HPC outlet Static pressure) (psia)      20630 non-null  float64
 10  (Ratio of fuel flow to Ps30) (pps/psia)  20630

In [9]:
data.isnull().sum()

engine                                     0
cycle                                      0
setting_1                                  0
setting_2                                  0
(LPC outlet temperature) (◦R)              0
(LPT outlet temperature) (◦R)              0
(bypass-duct pressure) (psia)              0
(HPC outlet pressure) (psia)               0
(Physical core speed) (rpm)                0
(HPC outlet Static pressure) (psia)        0
(Ratio of fuel flow to Ps30) (pps/psia)    0
(Bypass Ratio)                             0
(Bleed Enthalpy)                           0
(High-pressure turbines Cool air flow)     0
(Low-pressure turbines Cool air flow)      0
RUL                                        0
dtype: int64

In [8]:
data.shape

(20630, 16)

In [9]:
# prepare entity 

from dataclasses import dataclass
from pathlib import Path 


@dataclass(frozen = True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    clean_data_dir: Path
    all_schema: dict

In [10]:
from src.Mlflow_Project.constants import *
from src.Mlflow_Project.utils.utility import FileOperations

In [11]:
# configuration manager class , will read all yaml files 

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = FileOperations.read_yaml(config_filepath)
        self.params = FileOperations.read_yaml(params_filepath)
        self.schema = FileOperations.read_yaml(schema_filepath)

        FileOperations.create_directories([self.config.artifacts_root])

    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS


        FileOperations.create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            clean_data_dir = config.clean_data_dir,
            all_schema=schema,
        
    )


        return data_validation_config

In [12]:
# Building componets 

import os 
from src.Mlflow_Project.__init__ import logger 

In [13]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config 


    # validating all columns 

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None 

            data = pd.read_csv(self.config.clean_data_dir)
            all_cols = list(data.columns)


            all_schema = self.config.all_schema.keys()


            for col in all_cols:
                if col not in all_schema:
                    validation_status = False 
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status: {validation_status}")

                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status: {validation_status}")


            return validation_status
        
        except Exception as e:
            raise e 

        

In [14]:
# initializing my pipeline 

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-10-31 10:11:38,207: INFO: yaml file: config\config.yaml loaded successfully]
[2023-10-31 10:11:38,212: INFO: yaml file: params.yaml loaded successfully]
[2023-10-31 10:11:38,220: INFO: yaml file: schema.yaml loaded successfully]
[2023-10-31 10:11:38,223: INFO: created directory at: artifacts]
[2023-10-31 10:11:38,243: INFO: created directory at: artifacts/data_validation]
